# World Happiness Analysis

### This dataset focuses on the happiness metrics of each country, and along with it some other world health statistics

## Data Collection

Source for original datasets: 
* https://www.kaggle.com/mathurinache/world-happiness-report?select=2020.csv 
* https://data.worldbank.org/indicator/SP.POP.TOTL
* https://data.worldbank.org/indicator/GB.XPD.RSDV.GD.ZS?view=chart
* https://data.worldbank.org/indicator/SE.XPD.TOTL.GD.ZS?end=2019&most_recent_year_desc=false&start=1970&view=chart

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [2]:
import warnings
warnings.filterwarnings('ignore')

First, the happiness dataset is loaded and the columns and their values are observed.

In [3]:
df = pd.read_csv('happiness/2019.csv')
df.head()

,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


The statistics being compared to happiness for each country are population, government expenditure on education, and government expenditure on research and development. Datasets are loaded for these.

In [4]:
population = pd.read_csv('population.csv')
population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0,NaN


In [5]:
govt_edu_expense = pd.read_csv('government_expenditure_education.csv')       
govt_edu_expense.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.11913,6.54906,6.50538,5.90521,5.97571,5.52415,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.46201,2.60420,3.45446,3.69522,3.25580,4.22836,4.05887,NaN,NaN,NaN
2,Angola,AGO,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.53930,NaN,3.43797,3.96209,3.61172,NaN,NaN,NaN
4,Andorra,AND,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,2.98706,NaN,2.50616,3.07421,3.28035,3.23707,3.21709,3.24672,3.15162,NaN


In [6]:
rd_expenditure = pd.read_csv('r&d_expenditure.csv')
rd_expenditure.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.03229,NaN,NaN,NaN,NaN
3,Albania,ALB,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning

Datasets are being incorporated from various sources that contain different most recent years for different information. Since the most recent year is not consistent among all the countries in these datasets, this helper function adds a new column that keeps track of the most recent information for that country and another column that keeps track of which year this may be.

In [7]:
'''
NOTE: Use the function be
low caefully. It modifies the dataframe passed in. 
'''
def getMostRecentDataInDF(df, 
                           beginning_year, 
                           ending_year, 
                           new_column_name="most_recent", 
                           new_index_column_name="most_recent_year",
                           keyColumn='country',
                           keepOnlyMain=False
                        ):
    for index, row in df.iterrows():
        for year in reversed(range(beginning_year, ending_year + 1)):
            if not pd.isnull(row[str(year)]):
                df.at[index, new_column_name] = df.loc[index, str(year)]
                df.at[index, new_index_column_name] = year
                break
                
    if keepOnlyMain:
        df.drop(df.columns.difference([keyColumn, new_column_name, new_index_column_name]), 1, inplace = True)
        

Columns are renamed to shorter, more appropriate names to work with. 

In [8]:
df.rename(columns = {'GDP per capita':'gdp_per_capita', 
                     'Social support':'support',
                     'Healthy life expectancy':'life_expectancy', 
                     'Freedom to make life choices':'freedom', 
                     'Perceptions of corruption':'corruption', 
                     'Score': 'l_score', 
                     'Country or region': 'country', 
                     'Generosity': 'generosity'}, inplace = True)
df.head()

,country,l_score,gdp_per_capita,support,life_expectancy,freedom,generosity,corruption
0,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


Countries are then ranked by their ladder score to determine which countries are relatively happier than others.

In [9]:
df["rank"] = df['l_score'].rank() 

Using the regions provided by the dataset for 2020, each country in this datasest is assigned a corresponding region. A new column is created for this.

In [10]:
df2 = pd.read_csv('happiness/2020.csv')
df2.rename(columns={"Country name": "country", "Regional indicator": "r_indicator"}, inplace=True)
df = pd.merge(df, df2[["country", "r_indicator"]], on="country")
df.head()

,country,l_score,gdp_per_capita,support,life_expectancy,freedom,generosity,corruption,rank,r_indicator
0,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,156.0,Western Europe
1,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,155.0,Western Europe
2,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,154.0,Western Europe
3,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,153.0,Western Europe
4,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,152.0,Western Europe


As explained above, the dataset containing government education expenditure for each country has different most recent years when data was collected. The helper function is used to resolve this.

In [11]:
getMostRecentDataInDF(govt_edu_expense, 1960, 2020) 
govt_edu_expense.drop(govt_edu_expense.columns.difference(['Country Name', 'most_recent', 'most_recent_year']), 1, inplace = True)
govt_edu_expense.head()

,Country Name,most_recent,most_recent_year
0,Aruba,5.52415,2016.0
1,Afghanistan,4.05887,2017.0
2,Angola,3.42132,2010.0
3,Albania,3.61172,2017.0
4,Andorra,3.15162,2019.0


The literacy and population datasets get their most recent year for each country in a similar fashion. The final 3 datasets (govt_edu_expense, population, literacy) are all merged into the main happiness dataset.

In [12]:
country_map = {'Country Name': 'country'}
govt_edu_expense.rename(columns=country_map, inplace = True)
population.rename(columns=country_map, inplace = True)
rd_expenditure.rename(columns=country_map, inplace = True)

getMostRecentDataInDF(population, 1960, 2020, new_column_name='est_population', new_index_column_name='last_population_data', keepOnlyMain=True)
getMostRecentDataInDF(rd_expenditure, 1960, 2018, new_column_name='est_research', new_index_column_name='last_research_data', keepOnlyMain=True)

df = df.merge(govt_edu_expense, on='country')
df = df.merge(population, on='country')
df = df.merge(rd_expenditure, on='country')

df.rename(columns={'most_recent': 'est_govt_education_spending',
                   'most_recent_year': 'last_govt_spending_data'},
          inplace=True
         )
df.head()

,country,l_score,gdp_per_capita,support,life_expectancy,freedom,generosity,corruption,rank,r_indicator,est_govt_education_spending,last_govt_spending_data,est_population,last_population_data,est_research,last_research_data
0,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,156.0,Western Europe,6.37621,2017.0,5520314.0,2019.0,2.77381,2018.0
1,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,155.0,Western Europe,7.81608,2017.0,5818553.0,2019.0,3.06408,2018.0
2,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,154.0,Western Europe,7.91198,2017.0,5347896.0,2019.0,2.06985,2018.0
3,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,153.0,Western Europe,7.65822,2017.0,361313.0,2019.0,2.02994,2018.0
4,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,152.0,Western Europe,5.17510,2017.0,17332850.0,2019.0,2.16374,2018.0


Finally, the cleaned datasets are saved to new csv files.

In [13]:
df.to_csv('happiness_final.csv')
govt_edu_expense.to_csv('government_expenditure_education_final.csv')
population.to_csv('population_final.csv')
rd_expenditure.to_csv('r&d_expenditure_final.csv')